In [3]:
# GET PARAMETERS
import os 
import sys
import pandas as pd
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_and_visu_non_recurrent import evaluate_config,get_ds,train_the_config,get_ds_without_shuffling_on_train_set
from plotting.plotting import error_per_station_calendar_pattern,gain_between_models
from examples.train_model_on_k_fold_validation import load_configuration

In [4]:
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from trainer import Trainer
import torch 

save_folder = 'save/K_fold_validation/training_with_HP_tuning'
add_name_id = ''
trial_id1 ='subway_in_STGCN_MSELoss_2025_01_20_14_27_20569'

args1,_ = load_configuration(trial_id1,load_config=True)

fold_to_evaluate = [args1.K_fold-1]
modification = {'shuffle':False,
                'data_augmentation':False }


# Load Data and Init Model:
ds1,_,_,_,_ =  get_ds(args_init=args1,modification = modification,fold_to_evaluate=fold_to_evaluate)
model1 = load_model(ds1, args1)


# Load Trained Weights 
model_param1 = torch.load(f"{save_folder}/best_models/{trial_id1}{add_name_id}_F{args1.K_fold}_fcomplete_dataset.pkl")
model1.load_state_dict(model_param1['state_dict'],strict=True)


# Load Trainer : 
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model1,args1)
trainer1 = Trainer(ds1,model1,args1,optimizer,loss_function,scheduler = scheduler)


>>>> Load best CONFIG
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
Invalid dates within this fold: 776

Init Dataset: 'torch.Size([7392, 40]) with 295680 Total nb of elements and 0 Nan values
Init U/Utarget size: torch.Size([4702, 40, 7])/torch.Size([4702, 40, 1]) Train/Valid/Test 2821 940 940

 ===== ERROR WITH prefetch_factor====  
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing
Invalid dates within this fold: 481

Init Dataset: 'torch.Size([4597, 40]) with 183880 Total nb of elements and 0 Nan values
Init U/Utarget size: torch.Size([2940, 40, 7])/torch.Size([2940, 40, 1]) Train/Valid/Test 1059 941 939

 ===== ERROR WITH prefetch_factor====  
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing
Model size: 0.000GB
number of total parameters: 71713
number of trainable parameters: 71713


## Get Metrics from trained model. Compared consistency with saved one : 

In [12]:
training_mode = 'test'
full_predict1,Y_true,_ = trainer1.testing(ds1.normalizer, training_mode =training_mode)
Y_true= Y_true.detach().clone().reshape(-1)    
full_predict1= full_predict1.detach().clone().reshape(-1)    
error_pred1 = ((Y_true - full_predict1)**2).mean()


df_metrics1 = pd.read_csv(f"{save_folder}/METRICS_{trial_id1}.csv")
display(df_metrics1)
print(f"MSE errror on {training_mode} by loading trained model : {'{:.2f}'.format(error_pred1.item())}")

,Unnamed: 0,mse,mae,mape,mse_complete_ds,mae_complete_ds,mape_complete_ds
0,valid,1232.723193,20.924171,36.295816,1154.631104,20.133240,33.492840
1,test,1342.555347,21.619902,34.936220,1339.509766,20.983244,31.023258


MSE errror on test by loading trained model : 1552.68


## Problème dans les sauvegarde. On relance un test, epochs validation = 2, et on voit ce qu'on peut en re-tirer:

In [1]:
# GET PARAMETERS
import os 
import sys
import pandas as pd
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    
from examples.train_model_on_k_fold_validation import train_model_on_k_fold_validation,load_configuration
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from trainer import Trainer
import torch 
from constants.paths import SAVE_DIRECTORY
from examples.train_and_visu_non_recurrent import get_ds

trial_id1 ='subway_in_STGCN_MSELoss_2025_01_20_14_27_20569'
save_folder = 'K_fold_validation/training_with_HP_tuning/re_validation'
add_name_id = 'TEST'
training_mode = 'test'

Training and Hyper-parameter tuning with Ray is not possible


In [14]:
epochs_validation = 20
# Re-train the model: 
args1,folds = load_configuration(trial_id1,True)

modification ={'keep_best_weights':True,
                'epochs':epochs_validation,
                'device':torch.device("cuda:1"),
                }

train_model_on_k_fold_validation(trial_id1,load_config =True,
                                    save_folder=save_folder,
                                    modification=modification,
                                    add_name_id=add_name_id)
# ...


>>>> Load best CONFIG

>>>> Load best CONFIG
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
Invalid dates within this fold: 776



Init Dataset: 'torch.Size([7392, 40]) with 295680 Total nb of elements and 0 Nan values
2821 train samples had been added thank to Data Augmentation
calendar_dayofweek data augmented by dupplication but not modified
calendar_hour data augmented by dupplication but not modified
Train/Target size:  torch.Size([5642, 40, 7]) torch.Size([5642, 40, 1])
Init U/Utarget size: torch.Size([4702, 40, 7])/torch.Size([4702, 40, 1]) Train/Valid/Test 5642 940 940

 ===== ERROR WITH prefetch_factor====  
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing
Invalid dates within this fold: 481

Init Dataset: 'torch.Size([4597, 40]) with 183880 Total nb of elements and 0 Nan values
1059 train samples had been added thank to Data Augmentation
calendar_dayofweek data augmented by dupplication but not modified
calendar_hour data augmented by dupplication but not modified
Train/Target size:  torch.Size([2118, 40, 7]) torch.Size([2118, 40

In [15]:
### Once the model with this config is re-trained, load the saved parameter and check the consistency of the results:
args1,_ = load_configuration(trial_id1,load_config=True)

fold_to_evaluate = [args1.K_fold-1]
modification = {'shuffle':False,
                'data_augmentation':False }

# Load Data and Init Model:
ds1,_,_,_,_ =  get_ds(args_init=args1,modification = modification,fold_to_evaluate=fold_to_evaluate)
model1 = load_model(ds1, args1)


# Load Trained Weights 
model_param1 = torch.load(f"{current_path}/{SAVE_DIRECTORY}/{save_folder}/best_models/{trial_id1}{add_name_id}_fcomplete_dataset.pkl")
model1.load_state_dict(model_param1['state_dict'],strict=True)

# Load Trainer : 
optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model1,args1)
trainer1 = Trainer(ds1,model1,args1,optimizer,loss_function,scheduler = scheduler)


full_predict1,Y_true,_ = trainer1.testing(ds1.normalizer, training_mode =training_mode)
Y_true= Y_true.detach().clone().reshape(-1)    
full_predict1= full_predict1.detach().clone().reshape(-1)    
error_pred1 = ((Y_true - full_predict1)**2).mean()


df_metrics1 = pd.read_csv(f"{current_path}/{SAVE_DIRECTORY}/{save_folder}/METRICS_{trial_id1}TEST.csv")
display(df_metrics1)
print(f"MSE errror on {training_mode} by loading trained model : {'{:.2f}'.format(error_pred1.item())}")


>>>> Load best CONFIG
>>>>Model: STGCN; K_fold = 6; Loss function: MSE 
Invalid dates within this fold: 776

Init Dataset: 'torch.Size([7392, 40]) with 295680 Total nb of elements and 0 Nan values
Init U/Utarget size: torch.Size([4702, 40, 7])/torch.Size([4702, 40, 1]) Train/Valid/Test 2821 940 940

 ===== ERROR WITH prefetch_factor====  
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing


Invalid dates within this fold: 481

Init Dataset: 'torch.Size([4597, 40]) with 183880 Total nb of elements and 0 Nan values
Init U/Utarget size: torch.Size([2940, 40, 7])/torch.Size([2940, 40, 1]) Train/Valid/Test 1059 941 939

 ===== ERROR WITH prefetch_factor====  
ValueError: prefetch_factor option could only be specified in multiprocessing.let num_workers > 0 to enable multiprocessing
Model size: 0.000GB
number of total parameters: 71713
number of trainable parameters: 71713


,Unnamed: 0,mse,mae,mape,VAR_mse,VAR_mae,VAR_mape,mse_complete_ds,mae_complete_ds,mape_complete_ds
0,valid,1355.812744,22.050191,37.273778,10327.583649,0.829619,7.010509,1451.416382,22.981207,37.514351
1,test,1441.176343,22.780031,35.824788,6613.209349,0.615906,25.654875,1508.671997,22.972397,32.952045


MSE errror on valid by loading trained model : 1451.42


In [16]:
training_mode = 'valid'
full_predict1,Y_true,_ = trainer1.testing(None, training_mode =training_mode)
Y_true= Y_true.detach().clone().reshape(-1)    
full_predict1= full_predict1.detach().clone().reshape(-1)    
error_pred1 = ((Y_true - full_predict1)**2).mean()


df_metrics1 = pd.read_csv(f"{current_path}/{SAVE_DIRECTORY}/{save_folder}/VALID_{trial_id1}TEST.csv")
display(df_metrics1)
print(f"MSE errror on {training_mode} by loading trained model : {'{:.6f}'.format(error_pred1.item())}")

,Unnamed: 0,fold0,fold1,fold2,fold3,fold4,mean,complete_dataset
0,0,0.002939,0.002472,0.002494,0.002592,0.002877,0.002675,0.002877


MSE errror on valid by loading trained model : 0.002877


## Load Weights

## Load l'entrainement, et vérifie que les métriques sont cohérentes : 

In [ ]:
from high_level_DL_method import load_model,load_optimizer_and_scheduler
from trainer import Trainer
import torch 

args1,_ = load_configuration(trial_id1,load_config=True)
fold_to_evaluate = [args1.K_fold-1]
modification = {'shuffle':False,
                'data_augmentation':False }

ds1,_,_,_,_ =  get_ds(args_init=args1,modification = modification,fold_to_evaluate=fold_to_evaluate)
model1 = load_model(ds1, args1)


optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model1,args1)
trainer1 = Trainer(ds1,model1,args1,optimizer,loss_function,scheduler = scheduler)



In [ ]:
#weights_id = subway_in_STGCN_MSELoss_2025_01_20_14_40_26118_F6_fcomplete_dataset
weights_id = 'subway_in_STGCN_MSELoss_2025_01_17_18_31_60147_F6_fcomplete_dataset'
for weights_id in ['subway_in_STGCN_MSELoss_2025_01_17_18_31_60147_F6_fcomplete_dataset',
                   'subway_in_STGCN_MSELoss_2025_01_20_14_40_26118_F6_fcomplete_dataset',
                   'subway_in_STGCN_MSELoss_2025_01_17_15_14_85172_F6_fcomplete_dataset',
                   'subway_in_STGCN_MSELoss_2025_01_06_08_04_25065_F6_fcomplete_dataset']:
    
    model_param1 = torch.load(f"save/K_fold_validation/training_with_HP_tuning/best_models/{weights_id}.pkl")
    model1.load_state_dict(model_param1['state_dict'],strict=True)

    optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model1,args1)
    trainer1 = Trainer(ds1,model1,args1,optimizer,loss_function,scheduler = scheduler)

    training_mode = 'test'
    full_predict1,Y_true,_ = trainer1.testing(ds1.normalizer, training_mode =training_mode)
    Y_true= Y_true.detach().clone().reshape(-1)    
    full_predict1= full_predict1.detach().clone().reshape(-1)    
    error_pred1 = ((Y_true - full_predict1)**2).mean()

    print(error_pred1)

In [ ]:
# Init
training_mode_to_visualise = ['test']
station = ['PAR','PER','GER','BON']
for training_mode in training_mode_to_visualise:
    min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
    limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
    fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                freq='1h',
                                                min_flow=min_flow,
                                                figsize = (30,5*len(station)),
                                                limit_percentage_error = limit_percentage_error,
                                                stations = station)


In [ ]:
trial_id2 ='subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_20_05_38_87836' #Concate Early & Late

args2,_ = load_configuration(trial_id2,load_config=True)

fold_to_evaluate = [args2.K_fold-1]
modification = {'shuffle':False,
                'data_augmentation':False }

ds2,_,_,_,_ =  get_ds(args_init=args2,modification = modification,fold_to_evaluate=fold_to_evaluate)
model2 = load_model(ds2, args2)

## Load Weights

In [ ]:
weights_id = 'subway_in_subway_out_STGCN_VariableSelectionNetwork_MSELoss_2025_01_20_07_23_40226_F6_fcomplete_dataset'
model_param2 = torch.load(f"save/K_fold_validation/training_with_HP_tuning/best_models/{weights_id}.pkl")
model2.load_state_dict(model_param2['state_dict'],strict=True)

optimizer,scheduler,loss_function = load_optimizer_and_scheduler(model2,args2)
trainer2 = Trainer(ds2,model2,args2,optimizer,loss_function,scheduler = scheduler)

## Gain de Model2 par rapport à Model1

In [ ]:
training_mode = 'test'
full_predict1,Y_true,_ = trainer1.testing(ds1.normalizer, training_mode =training_mode)
full_predict2,_,_ = trainer2.testing(ds2.normalizer, training_mode =training_mode)

Y_true= Y_true.detach().clone().reshape(-1)    
full_predict1= full_predict1.detach().clone().reshape(-1)    
full_predict2= full_predict2.detach().clone().reshape(-1)   

error_pred1 = ((Y_true - full_predict1)**2).mean()
error_pred2 = ((Y_true - full_predict2)**2).mean()

print(error_pred1,error_pred2)

In [ ]:
station = list(ds.spatial_unit) #['PAR','PER','GER','BON']
gain_between_models(trainer,trainer2,ds,ds2,training_mode,
                         metrics = ['mse','mae','mape'],
                        freq='1h',
                        index_matshow = 'day_date',
                        columns_matshow = 'hour',
                        min_flow = 20,
                        figsize = (30,5*len(station)),
                        limit_percentage_error = 50,
                        acceptable_error = 10,
                        stations = station)

In [ ]:
station = ['PAR','PER','GER','BON']
gain_between_models(trainer,trainer2,ds,ds2,training_mode,
                         metrics = ['mse','mae','mape'],
                        freq='1h',
                        index_matshow = 'day_date',
                        columns_matshow = 'hour',
                        min_flow = 20,
                        figsize = (30,5*len(station)),
                        limit_percentage_error = 50,
                        acceptable_error = 10,
                        stations = station)

In [ ]:
#'subway_in_STGCN_MSELoss_2025_01_06_08_00_94523',
#"subway_in_netmob_POIs_STGCN_VariableSelectionNetwork_MSELoss_2025_01_09_07_54_72902" # contient  Instagram, Google Maps, Deezer, WhatsApps, Twiteter, DL, UL
trial_id ='subway_in_STGCN_MSELoss_2025_01_20_14_27_20569'
args,_ = load_configuration(trial_id,load_config=True,epochs=None)

fold_to_evaluate = [0,args.K_fold-1]
modification = {'shuffle':False,
                'data_augmentation':False }

# Load Best Model of config 'trial_id':

trainer,ds,args,trial_id,df_loss = train_the_config(args,modification,fold_to_evaluate)
trainer,ds_no_shuffle = get_ds_without_shuffling_on_train_set(trainer,modification,args,fold_to_evaluate)

station = ['BEL','PER','PAR','GER','CHA']   # 'BON'  #'GER'
training_mode_to_visualise = ['test']#,'valid','train']
modification ={'keep_best_weights':True,
                'epochs':100,
                'validation_split_method' : 'forward_chaining_cv',
                'min_fold_size_proportion': 0.75,
                'train_prop':0.6,
                'valid_prop':0.2,
                'test_prop':0.2,

                'set_spatial_units':station,
        
                'data_augmentation': True, #True,  #False
                'DA_method':'interpolation', # 'noise' # 'interpolation
                'DA_moment_to_focus' : None, #[{'hours':[0,23],'weekdays':[1,3]}], # None
                }
if False:
    ds,args_modif,trial_id,save_folder,df_loss = get_ds(args.model_name,args.dataset_names,args.dataset_for_coverage,
                                                modification=modification,args_init=args,fold_to_evaluate=[args.K_fold-1])

(trainer,ds,ds_no_shuffle,args) = evaluate_config(args_best_model.model_name,args_best_model.dataset_names,args_best_model.dataset_for_coverage,
                                                station = station,
                                                modification=modification,
                                                training_mode_to_visualise=training_mode_to_visualise,
                                                args_init =args_best_model,
                                                fold_to_evaluate =[args_best_model.K_fold-1])

# Init
for training_mode in training_mode_to_visualise:
    min_flow = 20  # Minimal Flow considered for MAPE, otherwise set error = 0
    limit_percentage_error = 200 # 300% plus mauvais que quand on se sert du previous 
    fig,axes = error_per_station_calendar_pattern(trainer,ds,training_mode,metrics = ['mse','mae','mape','previous_value'],
                                                freq='1h',
                                                min_flow=min_flow,
                                                figsize = (30,5*len(station)),
                                                limit_percentage_error = limit_percentage_error,
                                                stations = station)

## Load and train model

In [ ]:
# GET PARAMETERS
import os 
import sys
import torch 
# Get Parent folder : 
current_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from examples.train_model_on_k_fold_validation import train_model_on_k_fold_validation,load_configuration


save_folder = 'K_fold_validation/training_with_HP_tuning/re_validation'
trial_id = 'subway_in_STGCN_MSELoss_2025_01_20_14_27_20569'
name_trial = 'TEST'
epochs_validation = 1
args,folds = load_configuration(trial_id,True)

modification ={'keep_best_weights':True,
                'epochs':epochs_validation,
                'device':torch.device("cuda:1"),
                }


config_diffs = {name_trial:{}}

                
for add_name_id,config_diff in config_diffs.items():
    config_diff.update(modification)
    train_model_on_k_fold_validation(trial_id,load_config =True,
                                        save_folder=save_folder,
                                        modification=config_diff,
                                        add_name_id=add_name_id)